# Register Client and Create Access Token Notebook
- Find detailed information about client registration and access tokens in this blog post: [Authentication to SAS Viya: a couple of approaches](https://blogs.sas.com/content/sgf/2021/09/24/authentication-to-sas-viya/)
- Additional access token information is found at the end of this notebook.


### Run the cells below and follow the resulting instructions.

# Import packages and create variables

In [18]:
import requests
import json
import os
import base64

# set/create variables
client_id="api.client"     # api.client
client_secret="api.secret" # api.secret
baseURL = "https://sasserver.sas.com" #replace sasserver with SAS server

# Create access token

In [19]:
# create authorization url
codeURL = baseURL + "/SASLogon/oauth/authorize?client_id=" + client_id + "&response_type=code"

# enccode client string
client_string = client_id + ":" + client_secret
message_bytes = client_string.encode('ascii')
base64_bytes = base64.b64encode(message_bytes)
base64_message = base64_bytes.decode('ascii')

# promt with instructions and entry for auth code
print(f"* Please visit the following site {codeURL} in an Incognito browser window")
print("* If provided a login prompt, add your SAS login credentials with SASAdministrator rights")
print("* Once authenticated, you'll be redirected to an authoriztion screen, check all of the boxes that appear")
print("* This will result in a short string of numbers and letters such as `VAxVFVEnKr`; this is your authorization code; copy the code")
code = input("Please enter the authoriztion code you generated through the previous instructions, and then press Enter: ")

# generate API call for access token
url = f"{baseURL}/SASLogon/oauth/token#authorization_code"
payload = "grant_type=authorization_code&code=" + code
headers = {
'Accept': 'application/json',
'Content-Type': 'application/x-www-form-urlencoded',
'Authorization': "Basic " + base64_message
}

# process the results
response = requests.request("POST", url, headers=headers, data=payload, verify=False)
access_token = json.loads(response.text)['access_token']
refresh_token = json.loads(response.text)['refresh_token']
print(json.dumps(response.json(), indent=4, sort_keys=True))

# Create access_token.txt file 
directory = os.getcwd()
with open(directory + '/access_token.txt', 'w') as f:
    f.write(access_token)
print('The access token was stored for you as ' + directory + '/access_token.txt')
     
# Create refresh_token.txt file 
directory = os.getcwd()
with open(directory + '/refresh_token.txt', 'w') as f:
    f.write(refresh_token)
print('The refresh token was stored for you as ' + directory + '/refresh_token.txt')

* Please visit the following site https://create.demo.sas.com/SASLogon/oauth/authorize?client_id=api.client&response_type=code in an Incognito browser window
* If provided a login prompt, add your SAS login credentials with SASAdministrator rights
* Once authenticated, you'll be redirected to an authoriztion screen, check all of the boxes that appear
* This will result in a short string of numbers and letters such as `VAxVFVEnKr`; this is your authorization code; copy the code
{
    "access_token": "eyJqa3UiOiJodHRwczovL2xvY2FsaG9zdC9TQVNMb2dvbi90b2tlbl9rZXlzIiwia2lkIjoibGVnYWN5LXRva2VuLWtleSIsInR5cCI6IkpXVCIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiJjYWY0N2ExYi0zNDEwLTQzZDQtYmI0My0wNjhlZDYwNzc3MDIiLCJzZXNzaW9uX3NpZyI6ImNkZDAxMjYxLTJlNzctNDkyMy1iNjRiLTIyZjU4YjY0NzkwMCIsInVzZXJfbmFtZSI6IkF2YS5LbGlzc291cmFzQHNhcy5jb20iLCJvcmlnaW4iOiJhenVyZSIsImlzcyI6Imh0dHA6Ly9sb2NhbGhvc3QvU0FTTG9nb24vb2F1dGgvdG9rZW4iLCJhdXRob3JpdGllcyI6WyJTQVNTY29yZVVzZXJzIiwiRGF0YUJ1aWxkZXJzIiwiVW5pdGVkIGluIFNURU0gSW50ZXJucy0yMDI

c:\Users\avklis\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'create.demo.sas.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## Notes on the access token
- The access token has a 12 hour time-to-live (ttl).
- Use the refresh token to generate a new access token.
- The refressh token has a 90 day ttl.
- The access_token is valid in this Notebook and is transferable to other notebooks and used for external API calls.

# Use the refresh token to generate a new access token

In [1]:
# enccode client string
client_string = client_id + ":" + client_secret
message_bytes = client_string.encode('ascii')
base64_bytes = base64.b64encode(message_bytes)
base64_message = base64_bytes.decode('ascii')

# copy resfresh token from txtfile
directory = os.getcwd()
file = open(directory + "/refresh_token.txt")
# read the file as a list
refresh_token = file.readlines()
# close the file
file.close()
print(refresh_token)
type(refresh_token)

url = f"{baseURL}/SASLogon/oauth/token#refresh_token"

payload = "grant_type=refresh_token&refresh_token=" + ' '.join(refresh_token)
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Accept': 'application/json',
  'Authorization': "Basic " + base64_message
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

# print(response.text)
# process the results
response = requests.request("POST", url, headers=headers, data=payload, verify=False)
access_token = json.loads(response.text)['access_token']
print(json.dumps(response.json(), indent=4, sort_keys=True))

# Create access_token.txt file 
directory = os.getcwd()
with open(directory + '/access_token.txt', 'w') as f:
    f.write(access_token)
print('The access token was stored for you as ' + directory + '/access_token.txt')

NameError: name 'client_id' is not defined

# Test connection to SAS Viya via SWAT

In [28]:
import swat
print(swat.__file__)
directory = r"local_path/to/cert.pem" #replace with directory in which certificate is stored locally on computer"
print(directory)
httpconn = swat.CAS("https://sasserver.sas.com/cas-shared-default-http" #replace sasserver with SAS server
                    , 
                    username=None, 
                    password=access_token, 
                    ssl_ca_list=directory+ #"filename of certificate"
                    ,
                    protocol="https")
httpconn

#Binary (doesn't work)
# import swat
# binconn = swat.CAS("20.81.59.109", protocol="cas", username=None, password=access_token)

c:\Users\avklis\AppData\Local\Programs\Python\Python312\Lib\site-packages\swat\__init__.py
C:/Users/avklis/peanut_project/final


<html>
<head><title>404 Not Found</title></head>
<body>
<center><h1>404 Not Found</h1></center>
<hr><center>nginx</center>
</body>
</html>



SWATError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import swat
binconn = swat.CAS("20.81.59.109", protocol="cas", username=None, password=access_token, ssl_ca_list= "path/to/cert.crt" #replace with path to certificate
                   )
binconn

# Viya Rest call from Python Notebook 

In [ ]:
# Viya call uses hostname
url = "https://sasserver.sas.com/reports/reports" #replace sasserver with SAS server

payload={}
headers = {
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False).json()

print(response)

# CAS REST call from Python Notebook

In [ ]:
# CAS call uses ip addr
url= "https://20.237.68.50:443/cas-shared-default-http/cas/sessions"


payload={}
headers = {
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False).json()

print(response)

### Calling Model Studio models

In [30]:
# Since it's not on Pipy, you will to install from git using pip:
!pip install git+https://github.com/sassoftware/sas-scoring-translator-python.git

# loading the package
import pysct

  Cloning https://github.com/sassoftware/sas-scoring-translator-python.git to c:\users\avklis\appdata\local\temp\pip-req-build-cqdb3isx
  Resolved https://github.com/sassoftware/sas-scoring-translator-python.git to commit 40358e64183e1fe40b74d514144f57711e3863b8
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for pysct: filename=pysct-0.0.3-py3-none-any.whl size=15574 sha256=48a564a6912fd850354220de76215ee5a4edd196098b4c89a05328f0b334bd42
  Stored in directory: C:\Users\avklis\AppData\Local\Temp\pip-ephem-wheel-cache-xitlvjc0\wheels\b7\4b\3f\7632d9b73c6428d4f924f2c9da810f705228ddae096fcb122c
Successfully built pysct


  Running command git clone --filter=blob:none --quiet https://github.com/sassoftware/sas-scoring-translator-python.git 'C:\Users\avklis\AppData\Local\Temp\pip-req-build-cqdb3isx'

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
#Logistic Regression
#FINAL_DATA_CLEANED is the name of the dataset in my SAS Drive used for these models

out = pysct.DS_translate(
                in_file = r"path/to/score/code",   #update with path to scoring code
                out_caslib = "casuser",
                out_castable = "FINAL_DATA_CLEANED",
                in_caslib = "public",
                in_castable = "FINAL_DATA_CLEANED"
)

The file was successfully written to dmcas_scorecode.py
